In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [2]:
# output_1.txt
activity_string = 'drinking'
database = 'postgres'
eating_strings = ["eating", "drinking"]
endpoint = 'msds694.cmxsootjz10m.us-west-2.rds.amazonaws.com'
files = './WISDM/*/*/'
n = 20
properties = {'user': 'students', 'password': 'msdsstudents'}
subject_id = 1613
table = 'activity_code'
url = 'jdbc:postgresql://%s/%s' % (endpoint, database)

# output_2.txt
# activity_string = 'walking'
# database = 'postgres'
# eating_strings = ["eating"]
# endpoint = 'msds694.cmxsootjz10m.us-west-2.rds.amazonaws.com'
# files = './WISDM/*/*/'
# n = 10
# properties = {'user': 'students', 'password': 'msdsstudents'}
# subject_id = 1600
# table = 'activity_code'
# url = 'jdbc:postgresql://%s/%s' % (endpoint, database)


def retrive_file_name(x):
    """Returns subject_id, sensor, device and an arry of readings"""
    file_name = x[0].split("/")[-1].split(".txt")[0]
    file_arg = file_name.split("_")
    return (file_arg[1], file_arg[2], file_arg[3], x[1])


def convert_to_integer(x):
    """Convert a value to integer"""
    try:
        return int(x)
    except ValueError:
        return None


def convert_to_float(x):
    """Convert a value to float"""
    try:
        return float(x)
    except ValueError:
        return None


def conver_to_string(x):
    """Convert a value to string"""
    try:
        return str(x)
    except ValueError:
        return None


def check_same_user(x):
    """
    Return subject_id in the file name
    that is same as subject_id in the content.
    """
    if (x is not None and x[0] == x[3]):
        return (x[0], x[1], x[2], x[4], x[5], x[6], x[7], x[8])


def return_no_none_rows(x):
    """Return True if all the readings are not None"""
    if (x is not None and
            x[0] is not None and x[1] is not None and x[1] is not None and
            x[2] is not None and x[3] is not None and x[4] is not None and
            x[5] is not None and x[6] is not None and x[7] is not None):
        # if(x[5] == 0 or x[6] == 0 or x[7] == 0):
        return True
    else:
        return False


def create_flat_rdd(x):
    """
    Returns subject_id, sensor, device and
    subject_id, activity_code, x, y, z readings
    """
    values = x[3].split(",")
    if len(values) == 6:
        return (convert_to_integer(x[0]),
                x[1],
                x[2],
                convert_to_integer(values[0]),
                conver_to_string(values[1]),
                convert_to_integer(values[2]),
                convert_to_float(values[3]),
                convert_to_float(values[4]),
                convert_to_float(values[5]))


def file_rdd(ss, files):
    """Create a pair RDD using wholeTextFiles"""
    return ss.sparkContext.wholeTextFiles(files)


def create_activity_df(ss, files_rdd, schema):
    """Create dataframe using the schema"""
    activity_data_rdd = files_rdd.mapValues(lambda x: x.split(";\n"))\
        .flatMapValues(lambda x: x)\
        .map(retrive_file_name)\
        .map(create_flat_rdd)\
        .map(check_same_user)\
        .filter(return_no_none_rows)

    return ss.createDataFrame(activity_data_rdd, schema)


# Question 1

In [3]:
ss = SparkSession.builder.config('spark.driver.extraClassPath','postgresql-42.2.18.jar')\
                        .config("spark.executor.memory", "5g")\
                        .config("spark.driver.memory", "5g").getOrCreate()


In [4]:
# endpoint = 'msds694.cmxsootjz10m.us-west-2.rds.amazonaws.com'
# database = 'postgres'
# table = 'activity_code'
# properties = {'user': 'students', 'password': 'msdsstudents'}
# url = 'jdbc:postgresql://%s/%s' % (endpoint, database)

In [5]:
activity_code = ss.read.jdbc(url=url, table=table, properties=properties)

In [6]:
num_distinct_act = activity_code.distinct().count()
print(num_distinct_act)

18


# Question 2

In [7]:
activity_code.orderBy('activity', ascending=False).show(
    truncate=False)  # Show the full name by using truncate

+---------------------------+----+
|activity                   |code|
+---------------------------+----+
|Writing                    |Q   |
|Walking                    |A   |
|Typing                     |F   |
|Standing                   |E   |
|Stairs                     |C   |
|Sitting                    |D   |
|Playing Catch w/Tennis Ball|O   |
|Kicking Soccer Ball        |M   |
|Jogging                    |B   |
|Folding Clothes            |S   |
|Eating Soup                |H   |
|Eating Sandwich            |L   |
|Eating Pasta               |J   |
|Eating Chips               |I   |
|Drinking from Cup          |K   |
|Dribblinlg Basketball      |P   |
|Clapping                   |R   |
|Brushing Teeth             |G   |
+---------------------------+----+



# Question 3

In [8]:
# eating_strings = ["eating", "drinking"]

In [9]:
def check_eating(x):
    tracker = 0
    for i in eating_strings:
        if i in x:
            tracker = tracker + 1
    if tracker >= 1:
        return True
    else:
        return False


# Register the function as UDF
check_eating_udf = udf(check_eating, BooleanType())   # From class example 3

In [10]:
eating_df = activity_code.withColumn('eating', check_eating_udf(lower(
    activity_code['activity']))).orderBy('eating', 'code', ascending=[False, True])

In [11]:
eating_df.printSchema()

root
 |-- activity: string (nullable = true)
 |-- code: string (nullable = true)
 |-- eating: boolean (nullable = true)



In [12]:
eating_df.show()

+--------------------+----+------+
|            activity|code|eating|
+--------------------+----+------+
|         Eating Soup|   H|  true|
|        Eating Chips|   I|  true|
|        Eating Pasta|   J|  true|
|   Drinking from Cup|   K|  true|
|     Eating Sandwich|   L|  true|
|             Walking|   A| false|
|             Jogging|   B| false|
|              Stairs|   C| false|
|             Sitting|   D| false|
|            Standing|   E| false|
|              Typing|   F| false|
|      Brushing Teeth|   G| false|
| Kicking Soccer Ball|   M| false|
|Playing Catch w/T...|   O| false|
|Dribblinlg Basket...|   P| false|
|             Writing|   Q| false|
|            Clapping|   R| false|
|     Folding Clothes|   S| false|
+--------------------+----+------+



# Question 4

In [13]:
schema = StructType([StructField('subject_id', IntegerType(), False),
                     StructField('sensor', StringType(), False),
                     StructField('device', StringType(), False),
                     StructField('activity_code', StringType(), False),
                     StructField('timestamp', LongType(), False),
                     StructField('x', FloatType(), False),
                     StructField('y', FloatType(), False),
                     StructField('z', FloatType(), False)])

In [14]:
# Load the data to rdds
files_rdd = file_rdd(ss, files)

In [15]:
# Create the spark dataframe
files_df = create_activity_df(ss, files_rdd, schema)
# files_df.show(5)

In [16]:
result4 = files_df.select('subject_id', 'sensor', 'device', 'activity_code').distinct(
).groupBy('subject_id', 'sensor', 'device').count().orderBy('subject_id', 'device', 'sensor')

result4.show(result4.count())

+----------+------+------+-----+
|subject_id|sensor|device|count|
+----------+------+------+-----+
|      1600| accel| phone|   18|
|      1600|  gyro| phone|   18|
|      1600| accel| watch|   18|
|      1600|  gyro| watch|   18|
|      1601| accel| phone|   18|
|      1601|  gyro| phone|   18|
|      1601| accel| watch|   18|
|      1601|  gyro| watch|   18|
|      1602| accel| phone|   18|
|      1602|  gyro| phone|   18|
|      1602| accel| watch|   18|
|      1602|  gyro| watch|   18|
|      1603| accel| phone|   18|
|      1603|  gyro| phone|   18|
|      1603| accel| watch|   18|
|      1603|  gyro| watch|   18|
|      1604| accel| phone|   18|
|      1604|  gyro| phone|   18|
|      1604| accel| watch|   18|
|      1604|  gyro| watch|   18|
|      1605| accel| phone|   18|
|      1605|  gyro| phone|   18|
|      1605| accel| watch|   18|
|      1605|  gyro| watch|   18|
|      1606| accel| phone|   18|
|      1606|  gyro| phone|   18|
|      1606| accel| watch|   18|
|      160

# Question 5

In [17]:
# activity_code.show()
# files_df.show(5)

In [18]:
joined_df = files_df.join(
    activity_code, files_df.activity_code == activity_code.code).cache()

In [19]:
# joined_df.show(5)

In [20]:
selected_joined_df = joined_df.select(
    'subject_id', 'activity', 'device', 'sensor', 'x', 'y', 'z', 'activity_code').cache()

In [21]:
# selected_joined_df.show(5)

In [22]:
selected_joined_df.groupBy('subject_id','activity','device','sensor').\
                    agg(min('x').alias('x_min'),\
                        min('y').alias('y_min'),\
                        min('z').alias('z_min'),\
                        avg('x').alias('x_avg'),\
                        avg('y').alias('y_avg'),\
                        avg('z').alias('z_avg'),\
                        max('x').alias('x_max'),\
                        max('y').alias('y_max'),\
                        max('z').alias('z_max'),\
                        expr('percentile(x, array(0.05))')[0].alias('x_05%'),\
                        expr('percentile(y, array(0.05))')[0].alias('y_05%'),\
                        expr('percentile(z, array(0.05))')[0].alias('z_05%'),\
                        expr('percentile(x, array(0.25))')[0].alias('x_25%'),\
                        expr('percentile(y, array(0.25))')[0].alias('y_25%'),\
                        expr('percentile(z, array(0.25))')[0].alias('z_25%'),\
                        expr('percentile(x, array(0.50))')[0].alias('x_50%'),\
                        expr('percentile(y, array(0.50))')[0].alias('y_50%'),\
                        expr('percentile(z, array(0.50))')[0].alias('z_50%'),\
                        expr('percentile(x, array(0.75))')[0].alias('x_75%'),\
                        expr('percentile(y, array(0.75))')[0].alias('y_75%'),\
                        expr('percentile(z, array(0.75))')[0].alias('z_75%'),\
                        expr('percentile(x, array(0.95))')[0].alias('x_95%'),\
                        expr('percentile(y, array(0.95))')[0].alias('y_95%'),\
                        expr('percentile(z, array(0.95))')[0].alias('z_95%'),\
                        stddev('x').alias('x_std'),\
                        stddev('y').alias('y_std'),\
                        stddev('z').alias('z_std')).orderBy('activity','subject_id','device','sensor').show(n)

                        
            


+----------+--------------+------+------+-----------+-----------+-----------+--------------------+--------------------+--------------------+-----------+----------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+
|subject_id|      activity|device|sensor|      x_min|      y_min|      z_min|               x_avg|               y_avg|               z_avg|      x_max|     y_max|     z_max|               x_05%|               y_05%|               z_05%|               x_25%|               y_25%|               z_25%|               x_50%|               y_50%|               z_50%|               x_75%|               y_75%|               z_75%|               x_95%| 

# Question 6

In [23]:
extracted_joined = joined_df.select(
    'subject_id', 'activity', 'timestamp', 'device', 'sensor', 'x', 'y', 'z').cache()
# extracted_joined.show(5)

In [24]:
# def check_activity(x):
#     if activity_string in x:
#         return True
#     else:
#         return False
    
# # Register the function as UDF
# check_activity_udf = udf(check_eating, BooleanType()) 


In [25]:
extracted_joined.filter(f"subject_id=={subject_id}")\
                .orderBy('timestamp', 'device', 'sensor')\
                .filter(lower(extracted_joined['activity'])\
                        .contains(f"{activity_string}"))\
                .drop('subject_id').show(n)

+-----------------+---------------+------+------+------------+------------+------------+
|         activity|      timestamp|device|sensor|           x|           y|           z|
+-----------------+---------------+------+------+------------+------------+------------+
|Drinking from Cup|175766024086015| phone| accel|   1.0630256|   5.0996494|    8.136096|
|Drinking from Cup|175766024086015| phone|  gyro|-0.020240024|-0.004261058|-0.023435818|
|Drinking from Cup|175766059746973| phone| accel|   1.4281414|    5.017648|    7.902063|
|Drinking from Cup|175766059746973| phone|  gyro| -0.06844324| -0.14940333|-0.056459017|
|Drinking from Cup|175766099610941| phone| accel|   1.6065093|   5.3881507|    7.901464|
|Drinking from Cup|175766099610941| phone|  gyro|-0.049534798|-0.083889574|-0.059921127|
|Drinking from Cup|175766139549562| phone| accel|   1.2575544|   5.4102974|   7.9906483|
|Drinking from Cup|175766139549562| phone|  gyro| -0.07829694| -0.01917476| -0.03195793|
|Drinking from Cup|17

# Question 7

In [37]:
new_joined_df = joined_df.filter(f"subject_id=={subject_id}").filter(lower(extracted_joined['activity'])\
                        .contains(f"{activity_string}"))
#  new_joined_df.show(5)

+----------+------+------+-------------+---------------+------------+------------+------------+-----------------+----+
|subject_id|sensor|device|activity_code|      timestamp|           x|           y|           z|         activity|code|
+----------+------+------+-------------+---------------+------------+------------+------------+-----------------+----+
|      1613|  gyro| phone|            K|175766024086015|-0.020240024|-0.004261058|-0.023435818|Drinking from Cup|   K|
|      1613|  gyro| phone|            K|175766059746973| -0.06844324| -0.14940333|-0.056459017|Drinking from Cup|   K|
|      1613|  gyro| phone|            K|175766099610941|-0.049534798|-0.083889574|-0.059921127|Drinking from Cup|   K|
|      1613|  gyro| phone|            K|175766139549562| -0.07829694| -0.01917476| -0.03195793|Drinking from Cup|   K|
|      1613|  gyro| phone|            K|175766179413402|-0.061519023|  0.01438107| 0.007989483|Drinking from Cup|   K|
+----------+------+------+-------------+--------

In [39]:
# Filer for rows that has both sensor
both_sensor_df = new_joined_df.groupBy('activity_code', 'device', 'timestamp')\
    .agg(countDistinct('sensor').alias('sensor_count'))\
    .filter('sensor_count == 2')

In [40]:
both_sensor_df.show(5)

+-------------+------+---------------+------------+
|activity_code|device|      timestamp|sensor_count|
+-------------+------+---------------+------------+
|            K| phone|175766024086015|           2|
|            K| phone|175766059746973|           2|
|            K| phone|175766099610941|           2|
|            K| phone|175766139549562|           2|
|            K| phone|175766179413402|           2|
+-------------+------+---------------+------------+
only showing top 5 rows



In [41]:
extracted_df = joined_df.drop('activity', 'code')

In [42]:
extracted_df.show(5)

+----------+------+------+-------------+---------------+------------+------------+------------+
|subject_id|sensor|device|activity_code|      timestamp|           x|           y|           z|
+----------+------+------+-------------+---------------+------------+------------+------------+
|      1613|  gyro| phone|            K|175766024086015|-0.020240024|-0.004261058|-0.023435818|
|      1613|  gyro| phone|            K|175766059746973| -0.06844324| -0.14940333|-0.056459017|
|      1613|  gyro| phone|            K|175766099610941|-0.049534798|-0.083889574|-0.059921127|
|      1613|  gyro| phone|            K|175766139549562| -0.07829694| -0.01917476| -0.03195793|
|      1613|  gyro| phone|            K|175766179413402|-0.061519023|  0.01438107| 0.007989483|
+----------+------+------+-------------+---------------+------------+------------+------------+
only showing top 5 rows



In [43]:
# This dataframe only contains record of 'when both accelermeter and gyroscope exist for the same activity, device and timestamp'
big_joined_df = extracted_df.join(
      both_sensor_df, ['activity_code', 'device', 'timestamp'], 'leftsemi')

In [44]:
big_joined_df.show(5)

+-------------+------+---------------+----------+------+-------------+-------------+-------------+
|activity_code|device|      timestamp|subject_id|sensor|            x|            y|            z|
+-------------+------+---------------+----------+------+-------------+-------------+-------------+
|            K| phone|175770089551403|      1613|  gyro| -0.009321064|  -0.00479369|-0.0026631611|
|            K| phone|175770089551403|      1613| accel|    1.0313023|     5.122993|      8.10557|
|            K| phone|175774519993745|      1613|  gyro| -0.011185277|-0.0061252704|  5.326322E-4|
|            K| phone|175774519993745|      1613| accel|   0.96306765|     5.080496|     8.104373|
|            K| phone|175779032396139|      1613|  gyro|-0.0055926386|-0.0063915867|-0.0026631611|
+-------------+------+---------------+----------+------+-------------+-------------+-------------+
only showing top 5 rows



In [46]:
accel = big_joined_df.filter("sensor == 'accel'")\
    .filter(f"subject_id == {subject_id}")\
    .withColumnRenamed('x', 'accel_x')\
    .withColumnRenamed('y', 'accel_y')\
    .withColumnRenamed('z', 'accel_z')


gyro = big_joined_df.filter("sensor == 'gyro'")\
    .filter(f"subject_id == {subject_id}")\
    .withColumnRenamed('x', 'gyro_x')\
    .withColumnRenamed('y', 'gyro_y')\
    .withColumnRenamed('z', 'gyro_z')

In [47]:
accel.join(gyro, ['activity_code', 'device', 'timestamp'])\
     .orderBy('activity_code', 'timestamp')\
     .drop('subject_id', 'sensor')\
     .show(n)

+-------------+------+---------------+----------+---------+---------+-------------+------------+------------+
|activity_code|device|      timestamp|   accel_x|  accel_y|  accel_z|       gyro_x|      gyro_y|      gyro_z|
+-------------+------+---------------+----------+---------+---------+-------------+------------+------------+
|            K| phone|175766024086015| 1.0630256|5.0996494| 8.136096| -0.020240024|-0.004261058|-0.023435818|
|            K| phone|175766059746973| 1.4281414| 5.017648| 7.902063|  -0.06844324| -0.14940333|-0.056459017|
|            K| phone|175766099610941| 1.6065093|5.3881507| 7.901464| -0.049534798|-0.083889574|-0.059921127|
|            K| phone|175766139549562| 1.2575544|5.4102974|7.9906483|  -0.07829694| -0.01917476| -0.03195793|
|            K| phone|175766179413402| 1.0696096|5.1050367| 8.134899| -0.061519023|  0.01438107| 0.007989483|
|            K| phone|175766219317073| 1.2886791|5.1170077| 8.032547| -0.051931642| 0.015978966| 0.002396845|
|         

In [ ]:
ss.stop()